In [1]:
import pandas as pd
from EDA.OutlierHandling import OutlierHandling
from EDA.Encoding import Encoding
from EDA.NullHandling import NullHandling
from EDA.ColumnTypeIdentification import ColumnTypeIdentification
from EDA.FeatureReduction import FeatureReduction
import warnings
warnings.filterwarnings("ignore")
import threading
from Modelling.Classification import Classification
from sklearn.metrics import accuracy_score,f1_score
from itertools import product
from pathlib import Path

In [2]:
from MLAccelerator import MLAccelerator

In [3]:
from flask import Flask, render_template, request, redirect, url_for
import pandas as pd
app = Flask(__name__)

# render webpage
@app.route('/')
def home():
    return render_template('Home.html')


@app.route('/', methods=['POST', 'GET'])
def get_data():
    if request.method == 'POST':
        flag_list ={}
        final_list={}
        metric_dict={'accuracy_score':accuracy_score,'f1_score':f1_score}
        flag_list['outlier']=request.form.getlist('outlier')
        flag_list['nullhandle']=request.form.getlist('nullhandle')
        flag_list['feature']=request.form.getlist('feature')
        flag_list['encoding']=request.form.getlist('encoding')
        flag_list['metric']=request.form.getlist('metric')
        flag_list['model']=request.form.getlist('model')
        for key in flag_list:
            if not flag_list[key]:
                final_list[key] = [None]
            else:
                final_list[key] = flag_list[key]
        df = pd.read_csv(request.files.get('file'),delimiter=',')
        cols= list(df)
        return redirect(url_for('push_data', columns=cols))


# get the data for the requested query
@app.route('/push_data/<columns>')
def push_data(columns):
    return render_template('Home.html',data = columns)

            

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/May/2020 14:22:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2020 14:22:07] "GET /Home_files/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [19/May/2020 14:22:14] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [19/May/2020 14:22:14] "GET /push_data/Index%28%5B%27PassengerId%27%2C%20%27Survived%27%2C%20%27Pclass%27%2C%20%27Name%27%2C%20%27Sex%27%2C%20%27Age%27%2C%20%27SibSp%27%2C%0A%20%20%20%20%20%20%20%27Parch%27%2C%20%27Ticket%27%2C%20%27Fare%27%2C%20%27Cabin%27%2C%20%27Embarked%27%5D%2C%0A%20%20%20%20%20%20dtype%3D%27object%27%29 HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2020 14:22:14] "GET /push_data/Home_files/font-awesome.min.css HTTP/1.1" 404 -


In [2]:
import pandas as pd
df = pd.read_csv("train.csv")
list(df)

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import pandas as pd
from EDA.OutlierHandling import OutlierHandling
from EDA.Encoding import Encoding
from EDA.NullHandling import NullHandling
from EDA.ColumnTypeIdentification import ColumnTypeIdentification
from EDA.FeatureReduction import FeatureReduction
import warnings
warnings.filterwarnings("ignore")
import threading
from Modelling.Classification import Classification
from sklearn.metrics import accuracy_score,f1_score
from itertools import product
from pathlib import Path
import json 


# In[2]:


from MLAccelerator import MLAccelerator


# In[3]:


from flask import Flask, render_template, request, redirect, url_for
import pandas as pd
app = Flask(__name__)


# render webpage
@app.route('/')
def index():
    return render_template('index.html')


@app.route('/upload', methods=['POST', 'GET'])
def upload():
    if request.method == 'POST':
        df = pd.read_csv(request.files.get('file'),delimiter=',')
        df.to_csv('dataframe.csv',index= False)
        cols= list(df)
        return redirect(url_for('push_data', columns=cols))


@app.route('/push_data/<columns>')
def push_data(columns):
    columns = columns.replace('[','').replace(']','').replace("'",'').split(',')
    return render_template('Home.html',data = columns)


@app.route('/submit',methods=['POST', 'GET'])   
def submit():
    if request.method == 'POST':
        flag_list ={}
        final_list={}
        cols_list=[]
        metric_dict={'accuracy_score':accuracy_score,'f1_score':f1_score}
        flag_list['outlier']=request.form.getlist('outlier')
        flag_list['nullhandle']=request.form.getlist('nullhandle')
        flag_list['feature']=request.form.getlist('feature')
        flag_list['encoding']=request.form.getlist('encoding')
        flag_list['metric']=request.form.getlist('metric')
        flag_list['model']=request.form.getlist('model')
        cols_list = request.form.getlist('columns')
        for key in flag_list:
            if not flag_list[key]:
                final_list[key] = [None]
            else:
                final_list[key] = flag_list[key]
        print(final_list)
        y = request.form['target']
        df=pd.read_csv('dataframe.csv')
        ml=MLAccelerator(df[cols_list],y,final_list,metric_dict)
        result=ml.execute()
        result_final=[]
        for key in results:
            result_final.append(results[key])
        l=[]
        for dict_ in result_final:
            sample_=dict_
            d1={}
            i=0
            dict_1=sample_['Hyperparameter']
            d1['model']=dict_1['model']
            d1['score']=sample_['score']
            d1['log']=sample_['log']
            d1['log']
            l.append(d1)
        temp=0
        for d2 in l:
            if d2['score']>temp:
                temp=d2['score']
        l1=[]
        for d2 in l:
            if d2['score']==temp:
                d2['best_flag']='Yes'
            else:
                d2['best_flag']='No'
            l1.append(d2)
# In[ ]:

app.run()






In [ ]:
        y = request.form['target']
        ml=MLAccelerator(df,y,final_list,metric_dict)
        result=ml.execute()
        return result

In [2]:
import pandas as pd

In [10]:
data = pd.read_json(r'train_nlp.json')

In [11]:
data.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


In [12]:
data.to_csv('train_nlp.csv',index = False)

In [13]:
data1 = pd.read_csv('train_nlp.csv')

In [14]:
data1.head()

,id,cuisine,ingredients
0,10259,greek,"['romaine lettuce', 'black olives', 'grape tom..."
1,25693,southern_us,"['plain flour', 'ground pepper', 'salt', 'toma..."
2,20130,filipino,"['eggs', 'pepper', 'salt', 'mayonaise', 'cooki..."
3,22213,indian,"['water', 'vegetable oil', 'wheat', 'salt']"
4,13162,indian,"['black pepper', 'shallots', 'cornflour', 'cay..."
